In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

In [ ]:
!pip install "gensim>=4.0.0"

In [ ]:
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
import regex as re
# create tokenized documents
gensim_words = [[w for w in re.split(r'[\|\#]', doc.lower()) if w not in stop_words] 
                           for doc in df["nav"]]

In [ ]:
from gensim.corpora import Dictionary

dictionary = Dictionary(gensim_words) 

In [ ]:
dictionary.filter_extremes(no_below=5, no_above=0.7)

In [ ]:
bow = [dictionary.doc2bow(doc) for doc in gensim_words]

In [ ]:
from gensim.models import LdaModel
lda = LdaModel(corpus=bow, id2word=dictionary, num_topics=10, 
                      iterations=400, passes=20, random_state=42)

In [ ]:
lda.show_topics()

In [ ]:
from gensim.models import TfidfModel
tfidf_model = TfidfModel(bow)
tfidf = tfidf_model[bow]

In [ ]:
from gensim.models.nmf import Nmf
nmf = Nmf(corpus=tfidf, num_topics=10, id2word=dictionary, 
          w_max_iter=200,  h_max_iter=200, passes=20, random_state=42)

In [ ]:
nmf.show_topics()

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

lda_coherence = CoherenceModel(model=lda, texts=gensim_words, dictionary=dictionary, coherence='c_v')
lda_score = lda_coherence.get_coherence()
lda_score

In [ ]:
nmf_coherence = CoherenceModel(model=nmf, texts=gensim_words, dictionary=dictionary, coherence='c_v')
nmf_score = nmf_coherence.get_coherence()
nmf_score

In [ ]:
from tqdm.auto import tqdm
res = []
for num_topics in tqdm(range(5, 20)):
    lda = LdaModel(corpus=bow, id2word=dictionary, num_topics=num_topics, 
                      iterations=400, passes=20, random_state=42)
    lda_coherence = CoherenceModel(model=lda, texts=gensim_words, dictionary=dictionary, coherence='c_v')
    lda_score = lda_coherence.get_coherence()
    res.append({ "num_topics": num_topics, "score": lda_score })

In [ ]:
lda_num = pd.DataFrame(res)
lda_num.set_index("num_topics").plot()

In [ ]:
lda13 = LdaModel(corpus=bow, id2word=dictionary, num_topics=13, 
                      iterations=400, passes=20, random_state=42)

In [ ]:
lda13.show_topics()

In [ ]:
res = []
for num_topics in tqdm(range(5, 20)):
    nmf = Nmf(corpus=tfidf, num_topics=num_topics, id2word=dictionary, 
              w_max_iter=200,  h_max_iter=200, passes=20, random_state=42)
    nmf_coherence = CoherenceModel(model=nmf, texts=gensim_words, dictionary=dictionary, coherence='c_v')
    nmf_score = nmf_coherence.get_coherence()
    res.append({ "num_topics": num_topics, "score": nmf_score })

In [ ]:
nmf_num = pd.DataFrame(res)
nmf_num.set_index("num_topics").plot()

In [ ]:
nmf11 = Nmf(corpus=tfidf, num_topics=11, id2word=dictionary, 
            w_max_iter=200,  h_max_iter=200, passes=20, random_state=42)

In [ ]:
nmf11.show_topics()